In [8]:
from modules import process as prc
import pandas
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB


In [9]:
test_data = pandas.read_csv('../data/test.csv', sep=',', header=0)
train_data = pandas.read_csv('../data/train.csv', sep=',', header=0)

X_test = test_data.iloc[:, 0:5]
X_train = train_data.iloc[:, 0:5]
y_test = test_data.iloc[:, -1]
y_train = train_data.iloc[:, -1]

In [11]:
# bernoulli
BernNP = BernoulliNB(binarize = True)
BernNP.fit(X_train, y_train)

y_pred = BernNP.predict(X_test)

print(prc.accuracy_score(y_pred, y_test))


# Gaussian
gaussNB = GaussianNB()
gaussNB.fit(X_train, y_train)

y_pred = gaussNB.predict(X_test)

print(prc.accuracy_score(y_pred, y_test))

96.0
97.0
